# Vault Report (BMC Discovery)

This notebook reproduces the DisMAL `vault` report using the Tideway library only.
It checks the credential vault status and writes a single-line text file (`vault.txt`)
under the standard `output_<target>` folder.

## Requirements

Uncomment to install dependencies if needed.

In [ ]:
# %pip install -q tideway pandas pyyaml

import os, sys
from pathlib import Path
import pandas as pd
import yaml
from typing import Any


## Select Appliance (optional)

If `config.yaml` has multiple appliances, set by name or index. Defaults to the first.

In [ ]:
APPLIANCE_NAME = None   # e.g., 'prod' or 'dev'
APPLIANCE_INDEX = 0     # integer index if not using name selection


## Configuration (from config.yaml)

Loads target, token/token_file, API version, and SSL flag; prepares output folder.

In [ ]:
def _find_repo_root(start: Path) -> Path:
    for p in [start] + list(start.parents):
        if (p / 'config.yaml').exists():
            return p
    return start.parent

repo_root = _find_repo_root(Path.cwd())
config_path = repo_root / 'config.yaml'
with open(config_path, 'r') as fh:
    cfg = yaml.safe_load(fh) or {}

apps = cfg.get('appliances') or []
selected = None
if isinstance(apps, list) and apps:
    if APPLIANCE_NAME:
        selected = next((a for a in apps if a.get('name') == APPLIANCE_NAME), None)
        if selected is None:
            raise ValueError(f"No appliance named '{APPLIANCE_NAME}' in config.yaml")
    else:
        try:
            selected = apps[int(APPLIANCE_INDEX)]
        except Exception:
            selected = apps[0]

target = ((selected or {}).get('target') or cfg.get('target') or '').strip()
if not target:
    raise ValueError('config.yaml missing "target"')

token = (((selected or {}).get('token') or cfg.get('token') or '').strip())
token_file = (selected or {}).get('token_file') or cfg.get('token_file') or cfg.get('f_token')
if not token and token_file:
    tf_path = Path(token_file)
    if not tf_path.is_absolute():
        tf_path = repo_root / tf_path
    with open(tf_path, 'r') as tf:
        token = tf.read().strip()
if not token:
    raise ValueError('API token not found in config.yaml (token or token_file)')

API_VERSION = str((selected or {}).get('api_version') or cfg.get('api_version') or 'v1.14')
VERIFY_SSL = bool((selected or {}).get('verify_ssl', cfg.get('verify_ssl', True)))

sanitized = target.replace('.', '_').replace(':', '_').replace('/', '_')
output_dir = repo_root / f'output_{sanitized}'
output_dir.mkdir(parents=True, exist_ok=True)

print('Base Host     :', target)
print('API Version   :', API_VERSION)
print('Verify SSL    :', VERIFY_SSL)
print('Output folder :', output_dir)

# Prefer local Tideway package if present
local_tideway = repo_root / 'Tideway'
if local_tideway.exists():
    sys.path.insert(0, str(local_tideway))
import importlib
tideway = importlib.import_module('tideway')
API_VERSION_NUM = API_VERSION.lstrip('v')
app = tideway.appliance(target, token, api_version=API_VERSION_NUM, ssl_verify=VERIFY_SSL)
twvault = app.vault()
try:
    about = app.api_about
    print('Appliance reachable:', about.status_code)
except Exception as e:
    print('Warning: failed to contact appliance /api/about:', e)


## Helpers

Small utilities for safe JSON handling and status derivation.

In [ ]:
def get_json(resp_or_obj: Any):
    if hasattr(resp_or_obj, 'json'):
        try:
            return resp_or_obj.json()
        except Exception:
            return {}
    return resp_or_obj or {}

def derive_vault_status(info: dict) -> str:
    vopen = info.get('open')
    vsaved = info.get('passphrase_saved')
    vset = info.get('passphrase_set')
    status = 'OK'
    if not vopen:
        if not vsaved:
            status = 'Vault closed - Passphrase not saved'
    if not vset:
        status = 'Vault open - no passphrase set'
    return status


## Fetch Vault State

Calls `/vault` and previews the returned fields.

In [ ]:
resp = twvault.get_vault()
info = get_json(resp)
print('HTTP ok/status:', getattr(resp, 'ok', None), getattr(resp, 'status_code', None))
pd.DataFrame([info])


## Build Vault Output

Derives the same status string used by the CLI and displays it.

In [ ]:
vault_status = derive_vault_status(info or {})
print('Vault Status:', vault_status)
pd.DataFrame([[target, vault_status]], columns=['Discovery Instance','Vault Status'])


## Save to Text File

Writes a single-line `vault.txt` into the standard output folder.

In [ ]:
OUTPUT_TXT = output_dir / 'vault.txt'
with open(OUTPUT_TXT, 'w') as fh:
    fh.write(vault_status + '\n')
print(f'Saved to {OUTPUT_TXT}')
